### Guardador de dados
Série de scripts para processar as imagens em *jpg* que saem do *coletor_de_dados.py* e armazená-las, assim como os labels, em um arquivo binário, para posteriormente treinar a máquina.
Obs.: Cada conjunto de imagens que serão processadas a cada iteração deste notebook terão o mesmo label.

In [1]:
# Definição de variáveis

# MUDAR ESTAS
data_to_append = 20
data_labels = 1


data_path = "../data/data.npy"
labels_path = "../data/labels.npy"
data_dir = "../data/"


In [2]:
# Carrega as fotos já existentes

import numpy as np

try: 
    old_data = np.load(data_path)
    old_labels = np.load(labels_path)
    print("Files loaded sucessfuly.")
except FileNotFoundError:
    print("Files not found. Creating new ones...")
    with open(data_path, 'w'): pass
    with open(labels_path, 'w'): pass
    old_data = np.array([])
    old_labels = np.array([])


print("Data loaded: ", old_data.shape)


Files loaded sucessfuly.
Data loaded:  (708, 16, 16)


In [ ]:
# Carrega as fotos novas dos JPGs
import cv2
import matplotlib.pyplot as plt

new_data = np.array([])
for i in range(data_to_append):
    path = data_dir + f"data{i+1:03}.jpg"
    print("Loading image from", path)
    data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # Vamos mudar a imagem a fim de manter apenas o necessário
    # (não precisamos de 400x400 pixels)
    data = cv2.resize(data, (16, 16), interpolation=cv2.INTER_CUBIC) # Reduz o tamanho para (16, 16)
    _, data = cv2.threshold(data, 127, 1, cv2.THRESH_BINARY_INV) # Deixa a "tinta"=1 e fundo=0

    data = data.reshape((1, *data.shape))
    if not i:
        # O primeiro elemento define o 'shape' do array
        new_data = data
    else:
        # E os outros são adicionados no fim
        new_data = np.append(new_data, data, axis=0)

print(new_data.shape)

# Cria o vetor de novos labels
new_labels = np.array([data_labels for _ in range(len(new_data))])


In [ ]:
# Junto os dados

if len(old_data): # junta apenas se tiver dados antigos
    new_data = np.append(old_data, new_data, axis=0)
    new_labels = np.append(old_labels, new_labels, axis=0)


print("data size:", new_data.shape)
print("labels size:", new_labels.shape)



In [ ]:
# Finalização
# Coloca os novos dados no arquivo
np.save(data_path, new_data)
np.save(labels_path, new_labels)


In [ ]:
# Apaga as fotos
import os

for i in range(data_to_append):
    path = data_dir + f"data{i+1:03}.jpg"
    os.remove(path)
